In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, f1_score
import time

# Load a dummy dataset (MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Configure TensorFlow to allocate GPU memory on an as-needed basis
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Training loop with GPU VRAM tracking for the CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

total_vram_usage = 0
total_f1 = 0
num_checks = 5

# Expand dimensions for CNN input
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

# Train the CNN model
for epoch in range(num_checks):
    start_time = time.time()

    with tf.device('/GPU:0'):
        cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        cnn_model.fit(x_train, y_train, epochs=1, verbose=2)

        # Predict on the test set
        y_pred = cnn_model.predict(x_test)
        y_pred_classes = tf.argmax(y_pred, axis=1).numpy()

        # Calculate precision, recall, and F1 score
        precision = precision_score(y_test, y_pred_classes, average='weighted')
        recall = recall_score(y_test, y_pred_classes, average='weighted')
        f1 = f1_score(y_test, y_pred_classes, average='weighted')

        # Print information
        print(f"CNN Model, Epoch {epoch + 1}/5 - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    end_time = time.time()

    # Check GPU usage and extract VRAM usage
    gpu_info = !nvidia-smi
    vram_usage_line = next((line for line in gpu_info if 'MiB /' in line), None)

    if vram_usage_line:
        # Extract numerical parts and convert to integers
        vram_used_str = vram_usage_line.split()[8]
        vram_used = int(vram_used_str[:-3])  # Remove 'MiB' and convert to int

        # Print VRAM usage information
        print(f"VRAM Usage: {vram_used} MiB")

        total_vram_usage += vram_used
        total_f1 += f1

        # Optional: Add a delay to avoid overwhelming the GPU monitoring tool
        time.sleep(5)

# Calculate and print average VRAM usage
average_vram_usage = total_vram_usage / num_checks
average_f1 = total_f1 / num_checks
print(f"Average VRAM Usage: {average_vram_usage} MiB")
print(f"Average F1 Score: {round(average_f1, 2)}")


11490434/11490434 [==============================] - 0s 0us/step
1 Physical GPUs, 1 Logical GPUs
1875/1875 - 18s - loss: 0.1443 - accuracy: 0.9575 - 18s/epoch - 10ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 1/5 - Precision: 0.9806, Recall: 0.9806, F1 Score: 0.9806
VRAM Usage: 651 MiB
1875/1875 - 6s - loss: 0.0523 - accuracy: 0.9839 - 6s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 2/5 - Precision: 0.9830, Recall: 0.9829, F1 Score: 0.9829
VRAM Usage: 1163 MiB
1875/1875 - 5s - loss: 0.0325 - accuracy: 0.9899 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 2ms/step
CNN Model, Epoch 3/5 - Precision: 0.9860, Recall: 0.9859, F1 Score: 0.9859
VRAM Usage: 1163 MiB
1875/1875 - 5s - loss: 0.0206 - accuracy: 0.9938 - 5s/epoch - 3ms/step
313/313 [==============================] - 1s 3ms/step
CNN Model, Epoch 4/5 - Precision: 0.9867, Recall: 0.9867, F1 Score: 0.9867
VRAM Usage: 2187 MiB
1875/1875 - 7